# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [18]:
input_data='./input_data/'
output_data='./output_data/2020/'

In [15]:
model_name='Base Case'#MW153#MW144 'MW162'#'Fixed Tailwater'#'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [65]:
#models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
models=pd.read_csv(input_data + "2020/models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix
    
plant_capacity=model['Capacity']#180
canal_capacity=plant_capacity/180*1100#1100

model

OutputPrefix                       base
Description                   Base Case
EWRCatSet              Recommendation 1
HeadpondLift                          0
Capacity                            180
FixedTailwater                        0
Mean                                NaN
P95                                 NaN
P90                                 NaN
P75                                 NaN
P50                                 NaN
P25                                 NaN
P10                                 NaN
P5                                  NaN
StDev                               NaN
CoefVar                             NaN
Max                                 NaN
Min                                 NaN
CapFactor                           NaN
CalYearMean                         NaN
CalYearStDev                        NaN
CalYearCoefVar                      NaN
AutoCorrel                          NaN
Power_max                           NaN
CapFactor_Power_max                 NaN


## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [21]:
daily=pd.read_csv(output_data + 'ngonye_daily' + '_base_pe.csv') #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily.tail(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
Date,,,,,,,,,,,,,,,,,,,,,
2020-05-16,2020-05-27,3453.1404,1.021914,3528.810877,0.057,2020,5,16,2020.05,2019,...,0.98,0.98,0.98,31.271187,31.271187,31.271187,31.271187,125.084747,3002.033926,0.035447
2020-05-17,2020-05-28,3438.1311,1.025158,3524.628422,0.057,2020,5,17,2020.05,2019,...,0.98,0.98,0.98,31.295557,31.295557,31.295557,31.295557,125.182226,3004.373429,0.035516
2020-05-18,2020-05-29,3393.3391,1.031077,3498.793989,0.058,2020,5,18,2020.05,2019,...,0.98,0.98,0.98,31.446473,31.446473,31.446473,31.446473,125.785894,3018.861447,0.035951
2020-05-19,2020-05-30,3304.8137,1.041461,3441.835365,0.063,2020,5,19,2020.05,2019,...,0.98,0.98,0.98,31.781582,31.781582,31.781582,31.781582,127.126328,3051.031871,0.036936
2020-05-20,2020-05-31,3217.6933,1.034753,3329.517841,0.070,2020,5,20,2020.05,2019,...,0.98,0.98,0.98,32.452093,32.452093,32.452093,32.452093,129.808371,3115.400908,0.038987


In [44]:
daily=daily.drop(daily.loc['2020-05-01':].index)
daily.tail(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,ExceedanceRnd
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-26,2020-05-07,4444.4855,1.005485,4468.864852,0.026,2020,4,26,2020.04,2019,...,0.98,0.98,26.208560,26.208560,26.208560,26.208560,104.834241,2516.021780,0.023459,0.025
2020-04-27,2020-05-08,4427.0109,1.002491,4438.039767,0.027,2020,4,27,2020.04,2019,...,0.98,0.98,26.362291,26.362291,26.362291,26.362291,105.449163,2530.779915,0.023760,0.025
2020-04-28,2020-05-09,4392.1861,1.003235,4406.394760,0.028,2020,4,28,2020.04,2019,...,0.98,0.98,26.520924,26.520924,26.520924,26.520924,106.083696,2546.008710,0.024075,0.030
2020-04-29,2020-05-10,4374.8357,1.003235,4388.988231,0.028,2020,4,29,2020.04,2019,...,0.98,0.98,26.608535,26.608535,26.608535,26.608535,106.434138,2554.419320,0.024250,0.030
2020-04-30,2020-05-11,4305.8472,0.999115,4302.037140,0.031,2020,4,30,2020.04,2019,...,0.98,0.98,27.049967,27.049967,27.049967,27.049967,108.199869,2596.796857,0.025151,0.030


## Monthly


In [45]:

monthly=daily.groupby("MonthId").mean().drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly.tail(12)

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,Flow_difference,EWRChannelA,EWRChannelC,EWRChannelD,...,Power,PerfMwPerCumec,ExceedanceRnd,Power_max,Power_min,Turbines_max,Turbines_min,Energy,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
2019.05,977.418001,0.337290,2019.0,5.0,2018.0,8.0,11.935434,42.000000,9.000000,8.000000,...,158.596873,0.162285,0.336935,169.167338,141.295367,4,3,117996.073756,31,0.881094
2019.06,703.757761,0.437900,2019.0,6.0,2018.0,9.0,10.239927,43.000000,9.666667,8.333333,...,113.109198,0.159832,0.437500,144.809370,82.434378,3,2,81438.622350,30,0.628384
2019.07,433.649055,0.619903,2019.0,7.0,2018.0,10.0,6.964705,42.000000,9.000000,8.000000,...,63.868121,0.146203,0.620000,88.010523,45.197278,2,1,47517.881928,31,0.354823
2019.08,280.621323,0.819484,2019.0,8.0,2018.0,11.0,3.675001,37.225806,7.000000,5.000000,...,37.934713,0.134695,0.820000,49.980273,29.139409,1,1,28223.426554,31,0.210748
2019.09,190.090548,0.964633,2019.0,9.0,2018.0,12.0,2.336934,30.000000,6.000000,4.000000,...,25.466391,0.132939,0.964833,33.634176,18.164545,1,1,18335.801656,30,0.141480
2019.10,138.768157,0.995613,2019.0,10.0,2019.0,1.0,3.695914,24.000000,4.000000,2.000000,...,18.030974,0.128071,0.995806,25.615100,10.598074,1,1,13415.044375,31,0.100172
2019.11,207.487667,0.940867,2019.0,11.0,2019.0,2.0,2.509125,27.000000,4.000000,2.000000,...,31.984183,0.154181,0.941167,39.137988,27.178990,1,1,23028.611455,30,0.177690
2019.12,307.236929,0.778742,2019.0,12.0,2019.0,3.0,4.913177,29.225806,4.645161,3.000000,...,51.704364,0.166980,0.778871,70.763047,37.680629,2,1,38468.046761,31,0.287246
2020.01,510.932210,0.549097,2020.0,1.0,2019.0,4.0,7.819527,34.000000,5.000000,4.000000,...,88.865299,0.173407,0.549032,113.099430,64.501300,2,2,66115.782666,31,0.493696


## Yearly

In [46]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [48]:
yearly=yearly.drop(yearly.loc['2019':].index)
yearly.tail(5)

,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,,,,,,,,,
2014,1460.253290,208.780681,0.544560,0.099878,25.098362,18.623511,969.484512,964.706890,192.407900,25.316443,885112.640024,0.561335
2015,2975.524395,176.218173,0.494532,0.067347,25.390405,14.500960,973.562138,964.405522,191.268651,21.964965,785680.555059,0.498275
2016,3889.991232,156.550907,0.487279,0.047442,25.390405,12.642403,975.420695,964.405522,192.177553,18.622058,850630.836863,0.539467
2017,4719.752686,173.839355,0.556781,0.055665,25.390197,11.191615,976.871483,964.405522,188.355141,21.458716,840505.325413,0.533045
2018,1088.140256,159.330426,0.516120,0.113367,25.436464,20.516121,968.163947,964.405522,178.569683,18.164545,716766.357634,0.454570


In [49]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


## Power and energy exceedance

In [50]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed.head(3)

,Power
Exceedance,
0.00,193.908405
0.05,181.512866
0.10,172.999304


In [51]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc.head(3)

,Energy
0.00,1.019158e+06
0.05,9.575322e+05
0.10,9.207975e+05


In [52]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [53]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [54]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(180*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,853291.199211,0.541154
1926,778466.913536,0.493700
1927,854093.474919,0.541663
1928,826061.011044,0.523884
1929,754249.050837,0.478342


## Monthly Day
15th of every month

In [55]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [56]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-19,2020,4,19,2019,4486,0.25,2,70,14,18,...,0.85,0.85,0.85,0.85,26.1,26.1,26.1,26.1,104.5,2507
2020-04-20,2020,4,20,2019,4493,0.25,2,70,14,18,...,0.85,0.85,0.85,0.85,26.1,26.1,26.1,26.1,104.4,2504
2020-04-21,2020,4,21,2019,4525,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.9,25.9,25.9,25.9,103.7,2489
2020-04-22,2020,4,22,2019,4536,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.9,25.9,25.9,25.9,103.5,2483
2020-04-23,2020,4,23,2019,4525,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.9,25.9,25.9,25.9,103.7,2489
2020-04-24,2020,4,24,2019,4525,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.9,25.9,25.9,25.9,103.7,2489
2020-04-25,2020,4,25,2019,4486,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,26.1,26.1,26.1,26.1,104.5,2507
2020-04-26,2020,4,26,2019,4468,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,26.2,26.2,26.2,26.2,104.8,2516
2020-04-27,2020,4,27,2019,4438,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,26.4,26.4,26.4,26.4,105.4,2530


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [57]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

#if 'VicFalls' in weekly.columns:
weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(180*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  Flow_difference  EWRChannelA  \
WaterYear WaterWeek                                                          
2019      20          780.738134    0.403000        10.990097    36.000000   
          21          876.673726    0.367714        15.390902    36.000000   
          22         1180.574230    0.286000        67.454107    39.714286   
          23         2066.743151    0.172000       185.523336    49.000000   
          24         3377.689529    0.069000       176.837739    57.571429   
          25         4169.775452    0.033286        81.175645    59.000000   
          26         3945.721807    0.040571        29.681452    59.000000   
          27         3865.655413    0.043714        24.657355    68.428571   
          28         4004.023645    0.038143        31.251937    70.000000   
          29         4363.413687    0.028571        52.293971    70.000000   
          30         4500.873029    0.024286        20.345958    70.000000   
          31         4365.806710    0.029000        45.334209    70.000000   

                     EWRChannelC  EWRChannelD  EWRChannelE  EWRChannelFG  \
WaterYear WaterWeek                                                        
2019      20            6.000000     5.000000     4.000000     54.000000   
          21            6.000000     5.000000     4.000000     54.000000   
          22            6.857143     6.142857     5.428571     69.142857   
          23            9.000000     9.000000     9.000000    107.000000   
          24           10.714286    11.571429    12.428571    138.714286   
          25           11.000000    12.000000    13.000000    144.000000   
          26           11.000000    12.000000    13.000000    144.000000   
          27           13.571429    17.142857    23.285714    164.571429   
          28           14.000000    18.000000    25.000000    168.000000   
          29           14.000000    18.000000    25.000000    168.000000   
          30           14.000000    18.000000    25.000000    168.000000   
          31           14.000000    18.000000    25.000000    168.000000   

                       EWRTotal  EWRProportion  ...  HeadTurbine1_max  \
WaterYear WaterWeek                             ...                     
2019      20         105.000000       0.134616  ...         22.434289   
          21         105.000000       0.119917  ...         21.880315   
          22         127.285714       0.106584  ...         21.389000   
          23         183.000000       0.093966  ...         18.699794   
          24         231.000000       0.068992  ...         14.893494   
          25         239.000000       0.057341  ...         12.344726   
          26         239.000000       0.060581  ...         12.703626   
          27         287.000000       0.074244  ...         12.796033   
          28         295.000000       0.073694  ...         12.555854   
          29         295.000000       0.067658  ...         12.130262   
          30         295.000000       0.065547  ...         11.663395   
          31         295.000000       0.067578  ...         11.898434   

                     HeadTurbine1_min  LevelTailwater_max  LevelTailwater_min  \
WaterYear WaterWeek                                                             
2019      20                21.996824          967.095126          966.797591   
          21                21.271847          967.537344          967.171196   
          22                18.847637          969.282208          967.608975   
          23                15.134333          972.928765          969.410009   
          24                12.555854          975.507244          973.169604   
          25                11.928389          976.134709          975.718372   
          26                12.404897          975.658201          975.359472   
          27                12.564380          975.498718          975.267065   
          28                12.208243

In [58]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [59]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_median,Energy_min,Energy_P95,Energy_P90,Energy_P75,Energy_P25,Energy_P10,Energy_P5,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.578553,0.400982,0.399445,0.274923,120.0,93.882056,97.000000,52.0,...,20904.799105,6668.218097,16370.062348,17930.726516,18699.983313,26785.741369,31601.617808,36513.426172,7131.410088,0.308460
2,Nov,11,0.402395,0.333793,0.335532,0.218704,114.0,89.350694,93.000000,50.0,...,26701.362734,15892.504150,19486.470242,21547.023915,22835.535573,32109.353604,41048.888895,45499.257591,8971.628013,0.309506
3,Dec,12,0.288065,0.243408,0.243802,0.157836,126.0,100.264785,104.000000,59.0,...,47424.755866,28035.025183,36453.906988,38157.400713,42751.962128,60133.491782,68277.120212,84123.568843,14387.623781,0.277055
4,Jan,1,0.221764,0.179519,0.183488,0.097665,150.0,118.417003,120.000000,73.0,...,76677.540216,42262.422535,56464.444799,61346.255708,69081.776225,96684.202333,116073.372948,120644.491941,20484.919723,0.245852
5,Feb,2,0.188832,0.130423,0.139996,0.023961,197.0,139.262341,137.000000,85.0,...,98662.970820,4972.912035,72660.662032,77318.690923,89089.283655,110990.838808,119262.355015,121365.704729,18577.935412,0.189683
6,Mar,3,0.182992,0.095753,0.091030,0.040246,315.0,194.681452,201.064516,107.0,...,110672.323807,0.000000,22764.018419,77738.584338,95507.224732,121518.812380,129703.591106,132822.968791,29840.592086,0.288248
7,Apr,4,0.165295,0.085570,0.082382,0.049993,364.0,232.187500,217.000000,125.0,...,100014.509456,0.000000,59959.039235,73227.232831,83472.959336,115022.801386,129010.777693,133558.047400,26089.826456,0.265824
8,May,5,0.223150,0.130581,0.127581,0.066393,340.0,264.404414,276.000000,147.0,...,116679.318369,44299.068846,86448.892630,90812.806536,104434.445062,126389.484455,130875.610911,133657.904046,16515.802929,0.145198
9,Jun,6,0.368251,0.199430,0.195287,0.108731,293.0,212.859649,210.000000,119.0,...,113565.681109,32867.257737,50767.888847,70874.297260,95286.281066,121427.176477,123501.735735,124740.016721,23103.863569,0.220315


## Flow duration curve

In [60]:
daily['ExceedanceRnd']=np.round(daily['Exceedance']*2,2)/2
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,Flow_difference,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,EWRChannelE,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,8823.311921,0.001437,0.762334,136.886134,0.025775,1.098592,66.056338,12.408451,14.830986,17.183099,...,0.98,0.98,0.98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.005,6439.210565,0.005011,0.556348,113.817704,0.042514,1.102857,72.371429,13.834286,17.674286,22.565714,...,0.98,0.98,0.98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.010,5526.614698,0.009864,0.477500,72.510033,0.078352,1.301136,72.556818,13.607955,17.596591,22.937500,...,0.98,0.98,0.98,10.625409,10.625409,10.625409,10.625409,42.501637,1020.039277,0.007864
0.015,5158.859362,0.014978,0.445725,87.775999,0.101722,1.377778,72.038889,13.011111,16.811111,21.550000,...,0.98,0.98,0.98,23.021312,23.021312,23.021312,23.021312,92.085248,2210.045957,0.017867
0.020,4733.069287,0.020070,0.408937,110.624211,0.112690,1.415205,69.526316,13.175439,16.573099,21.163743,...,0.98,0.98,0.98,24.927678,24.927678,24.927678,24.927678,99.710711,2393.057067,0.021092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,182.773626,0.979641,0.015792,2.633446,0.893654,4.435897,28.589744,4.346154,2.788462,2.032051,...,NaN,NaN,NaN,25.123761,NaN,NaN,NaN,25.123761,602.970257,0.137431
0.985,174.604897,0.984725,0.015086,1.899215,0.907150,4.492228,27.834197,4.233161,2.601036,2.015544,...,NaN,NaN,NaN,23.770645,NaN,NaN,NaN,23.770645,570.495484,0.136173
0.990,164.118105,0.990257,0.014180,2.216955,0.942807,4.795322,26.105263,4.339181,2.467836,2.000000,...,NaN,NaN,NaN,22.475492,NaN,NaN,NaN,22.475492,539.411804,0.136906


## Add overall summary data to the model file

In [66]:

models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.at[model_name,'Power_max']=yearly['Power_max'].max()
models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

models.loc[model_name]



OutputPrefix                       base
Description                   Base Case
EWRCatSet              Recommendation 1
HeadpondLift                          0
Capacity                            180
FixedTailwater                        0
Mean                             830933
P95                              710170
P90                              751270
P75                              788598
P50                              838543
P25                              874970
P10                              920798
P5                               957532
StDev                             82123
CoefVar                       0.0988322
Max                         1.01916e+06
Min                              494343
CapFactor                      0.526975
CalYearMean                      831109
CalYearStDev                    78199.7
CalYearCoefVar                0.0940908
AutoCorrel                          NaN
Power_max                       193.908
CapFactor_Power_max             0.48928


In [62]:
autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
autocorrel

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1,0.475784,0.35078,0.291339,0.265053,0.15999,0.145133,0.0882527,0.171291,0.106375,...,-0.169674,-0.0946412,-0.00443844,0.0622853,0.0370406,-0.0236109,0.00356459,0.0132007,-0.0143362,0.0203947
1,"[1.0, 1.0]","[0.2746960056761163, 0.6768721294770477]","[0.10840907414625542, 0.5931507662705301]","[0.029242519411478374, 0.5534361796994004]","[-0.009827590773852035, 0.5399327978412047]","[-0.12503715096737322, 0.44501782800754763]","[-0.14350278444061995, 0.43376929650919505]","[-0.20331930397886339, 0.3798247315723354]","[-0.12135916228310964, 0.4639411805929794]","[-0.19030204749913993, 0.40305107036337207]",...,"[-0.5391755560929323, 0.19982828815487289]","[-0.4672803595196904, 0.2779979226783219]","[-0.3780482707597317, 0.3691713866010128]","[-0.31132669648188566, 0.4358972251435463]","[-0.33699102124620994, 0.41107218370149]","[-0.3977907739450812, 0.3505690259670762]","[-0.37067555130080043, 0.3778047275928327]","[-0.3610408373035355, 0.38744218739784814]","[-0.3885965097212052, 0.35992417078804206]","[-0.3538878382192626, 0.3946772524290646]"


## Save Files

In [63]:
outp=output_data+out_prefix

if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    fdc.to_csv(outp+'_pe_fdc.csv')
    autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    #models.to_csv(input_data + "models.csv")
    models.to_csv(input_data + "2020/models.csv")

In [64]:
years=daily['WaterYear'].unique().tolist()

for year in years:
    days=daily.loc[daily.WaterYear==year]
    days.to_csv(output_data + 'years/' + out_prefix + '_pe_daily_' + str(year) + '.csv')
 